In [1]:
!pip install selenium
!apt-get update
!apt install chromium-chromedriver

     |████████████████████████████████| 911kB 4.1MB/s 
Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [56.8 kB]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:10 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:11 http://archive.ubuntu.com/ubuntu bionic-updates In

In [2]:
from bs4 import BeautifulSoup
from selenium import webdriver
import re
import pandas as pd
from tqdm.notebook import tqdm
import random
import numpy as np
import time

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

In [6]:
# 장르별 base 주소
ballad = "https://www.melon.com/genre/song_list.htm?gnrCode=GN0100&dtlGnrCode=#params%5BgnrCode%5D=GN0100&params%5BdtlGnrCode%5D=&params%5BorderBy%5D=POP&params%5BsteadyYn%5D=N&po=pageObj&startIndex="
dance = "https://www.melon.com/genre/song_list.htm?gnrCode=GN0200#params%5BgnrCode%5D=GN0200&params%5BdtlGnrCode%5D=&params%5BorderBy%5D=POP&params%5BsteadyYn%5D=N&po=pageObj&startIndex="
hiphop = "https://www.melon.com/genre/song_list.htm?gnrCode=GN0300#params%5BgnrCode%5D=GN0300&params%5BdtlGnrCode%5D=&params%5BorderBy%5D=POP&params%5BsteadyYn%5D=N&po=pageObj&startIndex="
RB = "https://www.melon.com/genre/song_list.htm?gnrCode=GN0400#params%5BgnrCode%5D=GN0400&params%5BdtlGnrCode%5D=&params%5BorderBy%5D=POP&params%5BsteadyYn%5D=N&po=pageObj&startIndex="
indi = "https://www.melon.com/genre/song_list.htm?gnrCode=GN0500#params%5BgnrCode%5D=GN0500&params%5BdtlGnrCode%5D=GN0501&params%5BorderBy%5D=POP&params%5BsteadyYn%5D=N&po=pageObj&startIndex="
rock = "https://www.melon.com/genre/song_list.htm?gnrCode=GN0600#params%5BgnrCode%5D=GN0600&params%5BdtlGnrCode%5D=&params%5BorderBy%5D=POP&params%5BsteadyYn%5D=N&po=pageObj&startIndex="
trot = "https://www.melon.com/genre/song_list.htm?gnrCode=GN0700#params%5BgnrCode%5D=GN0700&params%5BdtlGnrCode%5D=GN0701&params%5BorderBy%5D=POP&params%5BsteadyYn%5D=N&po=pageObj&startIndex="
folk = "https://www.melon.com/genre/song_list.htm?gnrCode=GN0800#params%5BgnrCode%5D=GN0800&params%5BdtlGnrCode%5D=&params%5BorderBy%5D=POP&params%5BsteadyYn%5D=N&po=pageObj&startIndex="

In [ ]:
start = 1 #첫번째로 크롤링할 곡 번호(순위) 
num_of_data = 15000 #크롤링하고 싶은 데이터 갯수 
end = start + (num_of_data-50) + 1 #마지막으로 크롤링할 곡 번호(순위) 

In [ ]:
genre_dict = {ballad:'ballad', dance:'dance', hiphop:'rap/hiphop', RB: 'R&B/Soul', indi:'indi', 
             rock:'rock/metal', trot:'trot', folk:'folk/blues'}
genre_dict[trot]

'trot'

In [ ]:
df=pd.DataFrame(columns=['제목', '가수', '곡 번호'])

In [ ]:
def crawling(num_of_data, genre, start, end, df):
  for i in tqdm(range(start, end, 50)):
      driver = webdriver.Chrome('chromedriver', options=options) 
      url = genre + str(i)
      driver.get(url)
      #1초 내에서 랜덤으로 쉬어주기
      time.sleep(2)
      html = driver.page_source
      soup = BeautifulSoup(html, 'html.parser')    #html.parser를 사용해서 soup에 넣겠다
      
      #노래제목
      song_title = []
      #곡 번호
      song_number = []

      title = soup.find_all("div",{"class":"ellipsis rank01"})  
      for j in title:
        song_title.append(j.find('a').text)  #노래 제목 저장
        text = j.find('a')['href']
        song_number.append(re.search(r'[,](\d+)[)]', text).group(1)) # 곡 번호 저장
      
      
      #가수
      song_singer = []

      singer = soup.find_all("div",{"class":"ellipsis rank02"}) 
      for k in singer:
        try:
          song_singer.append(k.find('a').text) #가수 저장
        except AttributeError:
          song_singer.append('가수 확인 불가')

      #중간저장
      new = pd.DataFrame(np.array([song_title, song_singer, song_number]).T, columns=['제목', '가수', '곡 번호'])
      df = pd.concat([df, new], ignore_index=True)
      df.to_csv('/content/drive/Shareddrives/Parrot/Jinho/trot중간저장1-7500.csv', encoding='utf-8')
      print(str(i)+'부터 50개 저장완료')
      #2초 내에서 랜덤으로 쉬어주기
      time.sleep(0.5)
  
  return df
    


In [ ]:
crawling(num_of_data, trot, start, end, df)

1부터 50개 저장완료
51부터 50개 저장완료
101부터 50개 저장완료
151부터 50개 저장완료
201부터 50개 저장완료
251부터 50개 저장완료
301부터 50개 저장완료
351부터 50개 저장완료
401부터 50개 저장완료
451부터 50개 저장완료
501부터 50개 저장완료
551부터 50개 저장완료
601부터 50개 저장완료
651부터 50개 저장완료
701부터 50개 저장완료
751부터 50개 저장완료
801부터 50개 저장완료
851부터 50개 저장완료
901부터 50개 저장완료
951부터 50개 저장완료
1001부터 50개 저장완료
1051부터 50개 저장완료
1101부터 50개 저장완료
1151부터 50개 저장완료
1201부터 50개 저장완료
1251부터 50개 저장완료
1301부터 50개 저장완료
1351부터 50개 저장완료
1401부터 50개 저장완료
1451부터 50개 저장완료
1501부터 50개 저장완료
1551부터 50개 저장완료
1601부터 50개 저장완료
1651부터 50개 저장완료
1701부터 50개 저장완료
1751부터 50개 저장완료
1801부터 50개 저장완료
1851부터 50개 저장완료
1901부터 50개 저장완료
1951부터 50개 저장완료
2001부터 50개 저장완료
2051부터 50개 저장완료
2101부터 50개 저장완료
2151부터 50개 저장완료
2201부터 50개 저장완료
2251부터 50개 저장완료
2301부터 50개 저장완료
2351부터 50개 저장완료
2401부터 50개 저장완료
2451부터 50개 저장완료
2501부터 50개 저장완료
2551부터 50개 저장완료
2601부터 50개 저장완료
2651부터 50개 저장완료
2701부터 50개 저장완료
2751부터 50개 저장완료
2801부터 50개 저장완료
2851부터 50개 저장완료
2901부터 50개 저장완료
2951부터 50개 저장완료
3001부터 50개 저장완료
3051부터 50개 저장완료
3101부터 50개 저장완료
3151부터 50개 저장완료

,제목,가수,곡 번호
0,이제 나만 믿어요,임영웅,32508053
1,별빛 같은 나의 사랑아,임영웅,33337651
2,HERO,임영웅,33036660
3,계단말고 엘리베이터,임영웅,31254010
4,미워요,임영웅,8302665
...,...,...,...
14995,아름다운 날들,오민우,33440272
14996,가지도 못하면서,정다한,33466627
14997,미워요 (MR),임영웅,32532303
14998,낭만에 대하여 (MR),김호중,32532304
